In [2]:
import json

car_group = dict()


k5 = dict()

k5["price"] = "5000"

k5["year"] = "2015"

car_group["K5"] = k5


avante = dict()

avante["price"] = "3000"

avante["year"] = "2014"

car_group["Avante"] = avante


#json 파일로 저장

with open('./test.json', 'w', encoding='utf-8') as make_file:

    json.dump(car_group, make_file, indent="\t")


# 저장한 파일 출력하기

with open('./test.json', 'r') as f:

    json_data = json.load(f)

print(json.dumps(json_data, indent="\t"))


{
	"K5": {
		"price": "5000",
		"year": "2015"
	},
	"Avante": {
		"price": "3000",
		"year": "2014"
	}
}


In [26]:
import fitz
import PIL.Image
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams
import os
import json
import PIL.Image
import io
import re


path = r'./os.pdf'
extract_data = {}
pdf = fitz.open(path)
counter = 1
# full-text -> image 순으로 페이지당으로 나옴
for i in range(len(pdf)):
    full_text_data = {}
    page = pdf[i]  # load page
    images = page.get_images()
    text = page.get_text('sentences')
    # for image in images:
    #     base_img = pdf.extract_image(image[0])
    #     image_data = base_img['image']
    #     img = PIL.Image.open(io.BytesIO(image_data))
    #     extension = base_img['ext']
    #     img.save(
    #         open(f"temp_dataset/split_pages/image{counter}.{extension}", "wb"))
    full_text_data["full-text"] = text
    extract_data[str(counter)] = full_text_data
    counter += 1

# line-text 페이지당으로 나옴       
for page_layout in extract_pages(path):
    data = []
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                for character in text_line:
                    if isinstance(character, LTChar):
                        Font_size = round(character.size)
            data.append({'size': Font_size, 'line-text': (element.get_text())})
    data.insert(0, extract_data[str(page_layout.pageid)])
    extract_data[str(page_layout.pageid)] = (data)

# print(extract_data)
with open('./sample.json', 'w', encoding='utf-8') as make_file:

    json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
print("fin")


fin


In [2]:
import pdfminer
from pdfminer.image import ImageWriter
from pdfminer.high_level import extract_pages

pages = list(extract_pages('os.pdf'))
page = pages[0]


def get_image(layout_object):
    if isinstance(layout_object, pdfminer.layout.LTImage):
        return layout_object
    if isinstance(layout_object, pdfminer.layout.LTContainer):
        for child in layout_object:
            return get_image(child)
    else:
        return None


def save_images_from_page(page: pdfminer.layout.LTPage):
    images = list(filter(bool, map(get_image, page)))
    iw = ImageWriter('output_dir')
    for image in images:
        iw.export_image(image)


save_images_from_page(page)

